In [ ]:
import albumentations as A
import evaluate
import json
import numpy as np
import torch

from datasets import load_dataset
from huggingface_hub import hf_hub_download
from PIL import Image as PImage
from torch import nn
from transformers import MaskFormerForInstanceSegmentation, MaskFormerImageProcessor
from transformers import Trainer, TrainingArguments

In [ ]:
palette = [
    [120, 120, 120], [4, 200, 4], [180, 120, 120], [6, 230, 230],
    [80, 50, 50], [120, 120, 80], [140, 140, 140], [204, 5, 255]
]


def np_from_tensor(img_t):
    return np.moveaxis(img_t.numpy().astype(np.uint8), 0, -1)


def add_segmentations_to_image(img, segs):
    color_segmentation_map = np.zeros((segs.shape[0], segs.shape[1], 3), dtype=np.uint8)
    for label, color in enumerate(palette):
        color_segmentation_map[segs == label, :] = color
    img_mask = np.array(img) * 0.5 + color_segmentation_map * 0.5
    return img_mask.astype(np.uint8)


def mask_from_label(masks, labels, label_name):
  print("Label:", label_name)
  idx = labels.index(label_name)

  visual_mask = (masks[idx].bool().numpy() * 255).astype(np.uint8)
  return visual_mask


def add_mask_label_to_image(img, mask_label, label_idx):
    img_mask_label = np.zeros((mask_label.shape[0], mask_label.shape[1], 3), dtype=np.uint8)
    img_mask_label[mask_label == 255, :] = palette[label_idx]
    img_mask_label = 0.5 * img + 0.5 * img_mask_label
    return img_mask_label.astype(np.uint8)


In [ ]:
dataset_id = f"thiagohersan/satellite-trees"
base_model_id = f"facebook/maskformer-swin-base-ade"
result_model_id = f"maskformer-satellite-trees"

In [ ]:
id2label = json.load(open(hf_hub_download(dataset_id, "id2label.json", repo_type="dataset"), "r"))
id2label = {int(k):v for k,v in id2label.items()}
label2id = {v:int(k) for k,v in id2label.items()}

In [ ]:
model = MaskFormerForInstanceSegmentation.from_pretrained(
    base_model_id,
    id2label=id2label,
    ignore_mismatched_sizes=True
)

### Download Dataset

In [ ]:
dataset = load_dataset(dataset_id)
dataset = dataset.shuffle(seed=101010)
dataset = dataset["train"].train_test_split(test_size=0.2)

train_ds = dataset["train"]
test_ds = dataset["test"]

### Create MaskFormer Dataset

In [ ]:
preprocessor = MaskFormerImageProcessor(
    ignore_index=0,
    reduce_labels=False,
    do_resize=False,
    do_rescale=False,
    do_normalize=False
)

In [ ]:
def get_transform(transform=None):
    def apply_transform(batch_in):
        if transform is not None:
            img_labels = zip(batch_in["pixel_values"], batch_in["label"])
            aug_img_labels = [transform(image=np.array(x), mask=np.array(y)) for x,y in img_labels]
            images = [aug["image"] for aug in aug_img_labels]
            labels = [aug["mask"] for aug in aug_img_labels]
        else:
            images = [np.array(x) for x in batch_in["pixel_values"]]
            labels = [np.array(x) for x in batch_in["label"]]

        batch_out = preprocessor(images=images, segmentation_maps=labels, return_tensors="pt")
        return batch_out
    return apply_transform

In [ ]:
train_transform = A.Compose([
    A.CLAHE(),
    A.RandomRotate90(),
    A.Transpose(),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.75),
    A.Blur(blur_limit=3),
    A.OpticalDistortion(),
    A.GridDistortion(),
    A.HueSaturationValue(),
])

# train_ds.set_transform(get_transform(train_transform))
train_ds.set_transform(get_transform())
test_ds.set_transform(get_transform())

### Check Data

In [ ]:
example = train_ds[0]

for k,v in example.items():
  try:
    print(k,v.shape)
  except:
    print(f"{k}[0]",v[0].shape)

ex_labels = [id2label[label] for label in example["class_labels"].tolist()]
print(ex_labels)

In [ ]:
PImage.fromarray(np_from_tensor(example['pixel_values']))

In [ ]:
PImage.fromarray(mask_from_label(example["mask_labels"], ex_labels, 'tree'))

In [ ]:
PImage.fromarray(
    add_mask_label_to_image(
        np_from_tensor(example['pixel_values']),
        mask_from_label(example["mask_labels"], ex_labels, 'tree'),
        ex_labels.index('tree')
    )
)

### Train

In [ ]:
training_args = TrainingArguments(
    output_dir=f"{result_model_id}-outputs",
    learning_rate=5e-5,
    num_train_epochs=50,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    remove_unused_columns=False,
    report_to="tensorboard",
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_model_id=result_model_id,
    hub_strategy="end",
)

In [ ]:
metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
  print("hello")
  print(eval_pred)
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)

    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,
            reduce_labels=False)

    # add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
    per_category_iou = metrics.pop("per_category_iou").tolist()

    metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

    return metrics

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()